#### 0. LangChain에서 도구(tool) 활용 방법

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

gs
WA


In [2]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     #model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


client=<openai.resources.chat.completions.completions.Completions object at 0x0000021BC35DA540> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021BC37D7C80> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [4]:
# @tool 데코레이터로 선언된 함수는 직접 호출할 수 없다.
#add(10,20)

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [5]:
query = '3 곱하기 5는?'
llm_result = llm.invoke(query)

print(llm_result.content)

3 곱하기 5는 **15**입니다.  

계산 과정:  
\[ 3 \times 5 = 15 \]  

도움이 필요하시면 언제든지 물어보세요! 😊


- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [6]:
llm_with_tools = llm.bind_tools([add, multiply])

print(type(llm_with_tools))
print(llm_with_tools)

<class 'langchain_core.runnables.base.RunnableBinding'>
bound=ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021BC35DA540>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021BC37D7C80>, model_name='solar-pro', temperature=0.5, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1') kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': '숫자 a와 b를 더합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': '숫자 a와 b를 곱합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]} config={} config_factories=[]


- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [7]:
tool_result = llm_with_tools.invoke(query)

tool_result

AIMessage(content="[질문에 직접적으로 답하기 위해 'multiply' 함수를 호출해야 합니다. 이 함수는 3과 5를 곱하여 정확한 결과를 제공합니다.]", additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-dfc652ac035b4d8e8619d8b61866f71c', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 578, 'total_tokens': 624, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': 'c300fc23-8500-4783-98b7-d5e303c27c85', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--036f3613-c457-41e7-8292-2efb1e25d7a2-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'chatcmpl-tool-dfc652ac035b4d8e8619d8b61866f71c', 'type': 'tool_call'}], usage_metadata={'input_token

In [8]:
tool_result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'chatcmpl-tool-dfc652ac035b4d8e8619d8b61866f71c',
  'type': 'tool_call'}]

In [19]:
#query = "안녕하세요? 오늘의 날씨는 어떤가요?"
query = "10 더하기 5는 얼마인가요?"
tool_result = llm_with_tools.invoke(query)
print(tool_result.tool_calls)

[{'name': 'add', 'args': {'a': 10, 'b': 5}, 'id': 'chatcmpl-tool-5a34e237a44e43dba5f5b1323f1cae78', 'type': 'tool_call'}]


In [33]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

query = "20과 10을 곱한 결과는?"
human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 

print(message_list)


[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={})]


- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억하기

In [34]:
# llm_with_tools 는 LLM과 Tool이 연동된 RunnableBinding 객체
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='[질문에서 요구하는 "20과 10을 곱한 결과"를 얻기 위해 multiply 함수가 직접적으로 필요합니다. 다른 방법으로는 답을 구할 수 없으며, 이 함수 호출이 최소 요구사항에 부합합니다.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 582, 'total_tokens': 642, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': 'c734153c-b01d-4384-abe8-053313702dfb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ae727513-bf31-4d51-8244-87eee818c729-0', tool_calls=[{'name': 'multiply', 'args': {'a': 20, 'b': 10}, 'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d', 'type': 'tool_

In [35]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d',
  'type': 'tool_call'}]

In [36]:
from pprint import pprint

message_list.append(ai_message)

pprint(message_list)

[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[질문에서 요구하는 "20과 10을 곱한 결과"를 얻기 위해 multiply 함수가 직접적으로 필요합니다. 다른 방법으로는 답을 구할 수 없으며, 이 함수 호출이 최소 요구사항에 부합합니다.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 582, 'total_tokens': 642, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': 'c734153c-b01d-4384-abe8-053313702dfb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ae727513-bf31-4d51-8244-87eee818c729-0', tool_calls=[{'name': 'multiply', 'args': {'a

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [37]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d',
  'type': 'tool_call'}]

In [38]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

print(tool_message)

content='200' name='multiply' tool_call_id='chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d'


- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [39]:
message_list.append(tool_message)

pprint(message_list)

[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[질문에서 요구하는 "20과 10을 곱한 결과"를 얻기 위해 multiply 함수가 직접적으로 필요합니다. 다른 방법으로는 답을 구할 수 없으며, 이 함수 호출이 최소 요구사항에 부합합니다.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 582, 'total_tokens': 642, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': 'c734153c-b01d-4384-abe8-053313702dfb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ae727513-bf31-4d51-8244-87eee818c729-0', tool_calls=[{'name': 'multiply', 'args': {'a

In [40]:
tool_result = llm_with_tools.invoke(message_list)

pprint(tool_result)

AIMessage(content='20과 10을 곱한 결과는 **200**입니다.  \n\n계산 과정:  \n`20 × 10 = 200`  \n\n함수 호출 결과: `multiply(a=20, b=10) = 200`', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 658, 'total_tokens': 719, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '8ace0fb8-7f16-4f9f-865d-6e53eca6f080', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--4f7d6a95-d5d8-4375-9fb6-0bbbb1ae2753-0', usage_metadata={'input_tokens': 658, 'output_tokens': 61, 'total_tokens': 719, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


- `message_list`의 순서를 기억하기

In [41]:
message_list

[HumanMessage(content='20과 10을 곱한 결과는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[질문에서 요구하는 "20과 10을 곱한 결과"를 얻기 위해 multiply 함수가 직접적으로 필요합니다. 다른 방법으로는 답을 구할 수 없으며, 이 함수 호출이 최소 요구사항에 부합합니다.]', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-a9fb9e6350fa4b2c8231382cc5fc199d', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 582, 'total_tokens': 642, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': 'c734153c-b01d-4384-abe8-053313702dfb', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ae727513-bf31-4d51-8244-87eee818c729-0', tool_calls=[{'name': 'multiply', 'args': {'a